In [ ]:
category = 'Video_Games'
start_year=2015
end_year=2018

In [ ]:
%cd with_history

In [ ]:
import csv
def read_list_column_from_csv(train_file_path, column_name):
    result = []
    with open(train_file_path, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            list_str = row[column_name]
            list_obj = eval(list_str)
            result.append(list_obj)
    return result

train_file_path = f'./train/{category}_5_{start_year}-10-{end_year}-11.csv'
column_name = 'history_item_title'
list_of_lists = read_list_column_from_csv(train_file_path, column_name)

In [ ]:
item_to_inter = {}
for l in list_of_lists:
    for i in l:
        if i in item_to_inter:
            item_to_inter[i] += 1
        else:
            item_to_inter[i] = 1

In [ ]:
import json
with open(f'./info/{category}_title_interaction_5_{start_year}-10-{end_year}-11.json', mode='w', encoding='utf-8') as json_file:
    json.dump(item_to_inter, json_file, indent=4, ensure_ascii=False)

In [ ]:
from transformers import (
    AutoTokenizer
)
tokenizer = AutoTokenizer.from_pretrained('YOUR_MODEL_PATH', add_bos_token=False)
def get_hash(x):
    x = [str(_) for _ in x]
    return '-'.join(x)
def get_state_chain(x):
    x = [str(_) for _ in x]
    return '@'.join(x)

In [ ]:
try: 
    end_of_sentence_token_id = tokenizer.encode(
        "A sentence\n", add_special_tokens=False
    )[-1]
except:
    end_of_sentence_token_id = tokenizer.convert_tokens_to_ids("\n")

Default

In [ ]:
# n=1
import numpy as np
from collections import defaultdict
def nested_defaultdict():
    return defaultdict(float)
token_distri = defaultdict(nested_defaultdict)
for d,v in item_to_inter.items():
    for i in range(len(tokenizer.encode(d))):
        print(tokenizer.encode(d))
        if i==0:
            token_distri['first'][tokenizer.encode(d)[i]] += v
            if i==len(tokenizer.encode(d))-1:
                token_distri[str(tokenizer.encode(d)[i])][end_of_sentence_token_id] += v
        elif i==len(tokenizer.encode(d))-1:
            token_distri[get_hash(tokenizer.encode(d)[:i])][tokenizer.encode(d)[i]] += v
            token_distri[get_hash(tokenizer.encode(d))][end_of_sentence_token_id] += v
        else:
            token_distri[get_hash(tokenizer.encode(d)[:i])][tokenizer.encode(d)[i]] += v
token = {}
for k,v in token_distri.items():
    total_sum = sum(v.values())
    v = {key: np.log(value / total_sum) for key, value in v.items()}
    token[k] = v
with open(f'./info/{category}_token_distribution_5_{start_year}-10-{end_year}-11.json', mode='w', encoding='utf-8') as json_file:
    json.dump(token, json_file, indent=4, ensure_ascii=False)

Vary Supervision Granularity

In [ ]:
n=1 # 5 10 150

In [ ]:
# n
import json
import numpy as np
import math
from collections import defaultdict
def nested_defaultdict():
    return defaultdict(float)
token_distri = defaultdict(nested_defaultdict)
for d,v in item_to_inter.items():
    tokens = tokenizer.encode(d)
    state_num = math.ceil(len(tokens)/n)
    state_list = [get_hash(tokens[j*n:min(len(tokens),(j+1)*n)]) for j in range(state_num)]
    for i in range(state_num):
        current_state = state_list[i]
        if i==0:
            token_distri['first'][current_state] += v
            if i==state_num-1:
                token_distri[current_state][end_of_sentence_token_id] += v
        elif i==state_num-1:
            token_distri[get_state_chain(state_list[:i])][current_state] += v
            token_distri[get_state_chain(state_list)][end_of_sentence_token_id] += v
        else:
            token_distri[get_state_chain(state_list[:i])][current_state] += v
token = {}
for k,v in token_distri.items():
    total_sum = sum(v.values())
    v = {key: np.log(value / total_sum) for key, value in v.items()}
    token[k] = v
with open(f'./info/{category}_token_distribution_{n}_5_{start_year}-10-{end_year}-11.json', mode='w', encoding='utf-8') as json_file:
    json.dump(token, json_file, indent=4, ensure_ascii=False)

In [ ]:
def split_dict_into_groups(d, group_count=10):
    sorted_items = sorted(d.items(), key=lambda x: x[1], reverse=True)
    item_count = len(sorted_items)
    items_per_group = item_count // group_count
    remainder = item_count % group_count

    groups = []
    start_index = 0
    for i in range(group_count):
        end_index = start_index + items_per_group + (1 if i < remainder else 0)
        group = dict(sorted_items[start_index:end_index])
        groups.append(group)
        start_index = end_index
    return groups

groups = split_dict_into_groups(item_to_inter, 8)

for index, group in enumerate(groups):
    print(f"Group {index + 1}: {len(group)}")

In [ ]:
import json
group_title = {}
total_sum = 0
group_prob = []
for index, group in enumerate(groups):
    group_title[index] = list(group.keys())
    total_sum += sum(group.values())
for index, group in enumerate(groups):
    group_prob.append(sum(group.values())/total_sum)
print(group_prob)
with open(f'./info/{category}_group_title_8_{start_year}-10-{end_year}-11.json', mode='w', encoding='utf-8') as json_file:
    json.dump(group_title, json_file, indent=4, ensure_ascii=False)

In [ ]:
title_id = {}
with open(f'./info/{category}_5_{start_year}-10-{end_year}-11.txt', 'r') as f:
    for line in f:
        parts = line.strip().split('\t')
        title_id['\t'.join(parts[:-1])] = parts[-1]

group_ids = {}
for k,v in group_title.items():
    group_ids[k] = []
    for t in v:
        group_ids[k].append(title_id[t])
with open(f'./info/{category}_group_ids_8_{start_year}-10-{end_year}-11.json', mode='w', encoding='utf-8') as json_file:
    json.dump(group_ids, json_file, indent=4, ensure_ascii=False)